In [1]:
pip install newspaper3k serpapi transformers sentence-transformers

  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached feedparser-6.0.11-py3-none-any.whl.metadata (2.4 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 2.6 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
  Using cached sgmllib3k-1.0.0-py3-none-any.whl
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached feedparser-6.0.11-py3-none-any.whl (81 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Using cached soupsieve-2.7-

In [3]:
pip install lxml_html_clean

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
from newspaper import Article
from sentence_transformers import SentenceTransformer, util
import time

# 1. Setup Suchparameter (nutzt SerpAPI – Google ohne Scraping)
SERPAPI_KEY = "ba9c850db9303f0b69b9e456775aef08905b12c33e3da2b4e1e273b6dd5597c8"  # Kostenlos anmelden: serpapi.com
QUERY = "Volkswagen 100 Prozent Ökostrom EU Produktion 2023"
VERSPRECHEN = "Volkswagen nutzt seit 2023 in allen EU-Produktionsstätten 100 Prozent Ökostrom."

# 2. Lade lokales deutsches Sprachmodell
model = SentenceTransformer("distiluse-base-multilingual-cased")

# 3. Suche relevante Nachrichten
def suche_news(query, limit=5):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "tbm": "nws",
        "api_key": SERPAPI_KEY,
        "num": limit,
        "hl": "de",
        "gl": "de"
    }
    response = requests.get(url, params=params)
    ergebnisse = response.json().get("news_results", [])
    return [e["link"] for e in ergebnisse]

# 4. Artikel einlesen und Inhalte extrahieren
def lade_artikel(url):
    try:
        artikel = Article(url, language="de")
        artikel.download()
        artikel.parse()
        return artikel.text
    except Exception as e:
        print(f"Fehler beim Laden: {url} – {e}")
        return ""

# 5. Ähnlichkeit bewerten
def prüfe_versprechen(text, versprechen, threshold=0.7):
    doc_embedding = model.encode(text, convert_to_tensor=True)
    query_embedding = model.encode(versprechen, convert_to_tensor=True)
    score = float(util.pytorch_cos_sim(doc_embedding, query_embedding))
    return score >= threshold, score

# 6. Hauptlogik
links = suche_news(QUERY)
for link in links:
    print(f"\n🔗 {link}")
    inhalt = lade_artikel(link)
    if not inhalt:
        continue
    erfüllt, score = prüfe_versprechen(inhalt, VERSPRECHEN)
    if erfüllt:
        print(f"✅ Versprechen scheint eingehalten laut Artikel – Score: {score:.2f}")
    else:
        print(f"❌ Keine klare Bestätigung gefunden – Score: {score:.2f}")
    time.sleep(1)  # API freundlich bleiben

/Users/marieernst/Documents/Master/4. Semester/B5.3_Unternehmenssoftware/Greenwashing_Detector/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🔗 https://www.volkswagen-group.com/de/pressemitteilungen/anteil-erneuerbarer-energien-bei-der-stromversorgung-der-werke-steigt-deutlich-16750
❌ Keine klare Bestätigung gefunden – Score: 0.13


In [ ]:
import requests
from newspaper import Article
from sentence_transformers import SentenceTransformer, util
import time

#Konfiguration
SERPAPI_KEY = "ba9c850db9303f0b69b9e456775aef08905b12c33e3da2b4e1e273b6dd5597c8"  
QUERY = "Volkswagen 100 Prozent Ökostrom EU 2023"
VERSPRECHEN = "Volkswagen nutzt ab 2023 in allen EU-Produktionsstandorten ausschließlich Ökostrom."
MAX_ARTIKEL = 10
SIMILARITY_THRESHOLD = 0.7


def suche_news(query, max_results=20):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "tbm": "nws",
        "api_key": SERPAPI_KEY,
        "num": max_results,
        "hl": "de",
        "gl": "de"
    }
    response = requests.get(url, params=params)
    ergebnisse = response.json().get("news_results", [])
    return [e["link"] for e in ergebnisse]


def lade_artikel(url):
    try:
        artikel = Article(url, language="de")
        artikel.download()
        artikel.parse()
        return artikel.text
    except:
        return ""

#Ähnlichkeitsprüfung
model = SentenceTransformer("distiluse-base-multilingual-cased")

def prüfe_versprechen(text, versprechen, threshold=SIMILARITY_THRESHOLD):
    doc_emb = model.encode(text, convert_to_tensor=True)
    query_emb = model.encode(versprechen, convert_to_tensor=True)
    score = float(util.pytorch_cos_sim(doc_emb, query_emb))
    return score >= threshold, score


def finde_bestaetigung(query, versprechen, max_artikel=10):
    links = suche_news(query)
    geprüft = 0

    for i, link in enumerate(links):
        print(f"\n🔗 ({i+1}) {link}")
        text = lade_artikel(link)

        if len(text) < 500:
            print("Artikeltext zu kurz – übersprungen.")
            continue

        erfüllt, score = prüfe_versprechen(text, versprechen)
        print(f"Ähnlichkeitsscore: {score:.2f}")
        geprüft += 1

        if erfüllt:
            print(f"Versprechen bestätigt – Score: {score:.2f}")
            return link

        if geprüft >= max_artikel:
            print("Max. Artikelanzahl erreicht – keine Bestätigung gefunden.")
            break

        time.sleep(1)  # höflich bleiben bei Anfragen

    return None

# ▶️ Ausführung
link = finde_bestaetigung(QUERY, VERSPRECHEN)
if link:
    print(f"\n Bestätigung gefunden: {link}")
else:
    print("\n Keine Bestätigung in den geprüften Artikeln gefunden.")



🔗 (1) https://www.volkswagen-group.com/de/pressemitteilungen/anteil-erneuerbarer-energien-bei-der-stromversorgung-der-werke-steigt-deutlich-16750
🔎 Ähnlichkeitsscore: 0.17

❌ Keine Bestätigung in den geprüften Artikeln gefunden.
